In [95]:
import copy
import spacy
nlp_en = spacy.load('en')
nlp_de = spacy.load('de')

In [142]:
training_csv = None
with open('textProcessing_trainingKaldi.csv', 'r')  as file_csv:
    training_csv = file_csv.readlines()

In [143]:
def count_incorrect_meaning_language(data_set):
    lang_incorrect = {}
    meaning_incorrect = {}
    for sample in data_set:
        split = sample.split("\t")
        key = split[1]
        language = split[4]
        meaning = split[5].replace("\n", "")
              
        if language == "incorrect":
            if key in lang_incorrect:
                lang_incorrect[key] += 1
            else:
                lang_incorrect[key] = 1
        if meaning == "incorrect":
            if key in meaning_incorrect:
                meaning_incorrect[key] += 1
            else:
                meaning_incorrect[key] = 1
            
    return lang_incorrect, meaning_incorrect

lang_incorrect, meaning_incorrect = count_incorrect_meaning_language(training_csv[1:])

In [442]:
def prompt_as_map(split):
    rec_result = split[2]
    transcript = split[3]
    language = split[4]
    meaning = split[5].replace("\n", "")
    return {
        "rec_result": rec_result,
        "transcript": transcript,
        "language": language,
        "meaning": meaning
            }

def prompt_csv_map(data_set):
    rec_result_set = {}
    transcript_set = {}

    for sample in data_set:
        split = sample.split("\t")
        key = split[1]
        rec_result = split[2]
        transcript = split[3]
        language = split[4]
        meaning = split[5].replace("\n", "")
        item_map = prompt_as_map(split)
        if key in rec_result_set:
            rec_result_set[key].append(item_map)
        else:
            arr = []
            arr.append(item_map)
            rec_result_set[key] = arr

        if key in transcript_set:
            transcript_set[key].append(item_map)
        else:
            arr = []
            arr.append(item_map)
            transcript_set[key] = arr
    
    return rec_result_set, transcript_set

rec_result_set, transcript_set = prompt_csv_map(training_csv[1:])  

In [152]:
def create_pos_tag(nlp_sent, text):
    tag = ""
    pos = ""

    for word in nlp_sent:
        #sent_de += word.text + ' '
        tag += word.tag_ + ' '
        pos  += word.pos_ + ' '

    return_string =("%s \t %s \t %s" % (text, tag, pos))
    return return_string

def extract_pos_tag_rg(rg, output):
    tree = ET.parse(rg)
    grammar = tree.getroot()
    with open(output, "w") as writer:
        for prompt_unit in grammar:
            prompt_sent = None
            translated_sent = None
            counter = 0
            lang_incorrect_counter = 0
            meaning_incorrect_counter = 0
            for prompt in prompt_unit:
                if prompt.tag == None:
                    continue
                else:
                    if(prompt.tag == 'prompt'):
                        text = prompt.text
                        doc = text.replace('Frag: ','').replace('Sag: ','')
                        doc_de = nlp_de(doc)
                        prompt_sent = create_pos_tag(doc_de, text)
                        key = text
                        if key in lang_incorrect:
                            lang_incorrect_counter = lang_incorrect[key]
                        if key in meaning_incorrect:
                            meaning_incorrect_counter = meaning_incorrect[key]
                    elif(prompt.tag == 'translated_prompt'):
                        text = prompt.text
                        doc = text.replace("Ask for: ", '').replace("Say: ", '')
                        doc_en = nlp_en(doc)
                        translated_sent = create_pos_tag(doc_en, text)
                    elif(prompt.tag == 'response'):
                        counter += 1
            sent = ("%s \t %s \t %s \t %s \t %s \n" % (prompt_sent, translated_sent, str(counter), str(lang_incorrect_counter), str(meaning_incorrect_counter)))
            writer.write(sent)            
extract_pos_tag_rg("referenceGrammar.xml", "prompt_pos_tag.csv")

In [141]:
meaning_incorrect

{'Frag: 1 Musical-Ticket': 3,
 'Frag: 3 Tickets': 1,
 'Frag: 3 Tickets für Dienstagabend': 1,
 'Frag: 3 Tickets für König der Löwen': 2,
 'Frag: 3 Tickets für Wicked': 1,
 'Frag: 3 Tickets für morgen Abend': 1,
 'Frag: 3 Tickets nach London': 1,
 'Frag: 4 Tickets für König der Löwen': 1,
 'Frag: 4 Tickets für Wicked': 1,
 'Frag: 4 Tickets nach London': 1,
 'Frag: 5 Tickets nach London': 1,
 'Frag: Auberginen': 1,
 'Frag: Bohnen': 1,
 'Frag: Custard Tart': 1,
 'Frag: Doppelzimmer': 5,
 'Frag: Einzelzimmer': 9,
 'Frag: Erbsen': 4,
 'Frag: Gibt es ein Fitnessstudio?': 8,
 'Frag: Gibt es ein Hotelrestaurant?': 2,
 'Frag: Gibt es ein Schwimmbad?': 6,
 'Frag: Gibt es eine Hotelbar?': 5,
 'Frag: Gibt es einen Coiffeur?': 10,
 'Frag: Gibt es einen Lebensmittelladen?': 21,
 'Frag: Gibt es einen Lift?': 1,
 'Frag: Grösse 36': 1,
 'Frag: Grösse 40': 1,
 'Frag: Grösse 41': 1,
 'Frag: Grösse 42': 2,
 'Frag: Grösse 43': 1,
 'Frag: Grösse 44': 1,
 'Frag: Hosen': 5,
 'Frag: Ich möchte die Dessertkarte

## Reading textProcessingtraingFile

In [2]:
training_csv[0].replace('\t', ' ').replace('\n', '')

'Id Prompt RecResult Transcription language meaning'

In [3]:
training_csv[3].split('\t')

['5837',
 'Frag: Junior Suite',
 'single room',
 'kinderzimmer',
 'incorrect',
 'incorrect\n']

## Overview of File

In [4]:
def count_language_meaning():
    language_correct = 0
    language_false = 0
    meaning_correct = 0
    meaning_false = 0

    language_meaning_false = 0
    for line in training_csv[1:]:
        items = line.split('\t')
        language = items[4]
        meaning = items[5].replace('\n', '')
        if language == 'correct':
            language_correct += 1
        else:
            language_false += 1
        if meaning == 'correct':
            meaning_correct += 1
        else:
            meaning_false +=1

        if meaning == 'incorrect' and language == 'incorrect':
            language_meaning_false += 1
    return language_correct, language_false, meaning_correct, meaning_false, language_meaning_false

In [5]:
language_correct, language_false, meaning_correct, meaning_false, language_meaning_false = count_language_meaning()
print("Language correct: %s \t Language false: %s" % (language_correct, language_false))
print("Meaning correct: %s \t Meaning false: %s" % (meaning_correct, meaning_false))
print("Intersection between false Language and false Meaning: %s " % language_meaning_false)

Language correct: 3880 	 Language false: 1342
Meaning correct: 4682 	 Meaning false: 540
Intersection between false Language and false Meaning: 540 


## Recreate Language and Meaning File

In [6]:
import xml.etree.ElementTree as ET

In [416]:
def create_prompt_response_map(gram):
    prompt_response_map = {}
    for prompt_unit in gram:
        key = None
        responses = []
        for prompt in prompt_unit:
            if prompt.tag == None:
                print("None")
            else:
                if(prompt.tag == 'prompt'):
                    key = prompt.text
                elif(prompt.tag == 'response'):
                    responses.append(prompt.text)
        prompt_response_map[key] = responses
    return prompt_response_map

def read_grammar_and_create_map(file):
    tree = ET.parse(file)
    grammar = tree.getroot()
    return create_prompt_response_map(grammar)
    
#prompt_response_map = read_grammar_and_create_map('referenceGrammar.xml')
prompt_response_map_3 = read_grammar_and_create_map("man_rg.xml")
prompt_response_map_3

{'Sag: Ich gehe in die Ferien': ["i'm going on vacation",
  'i am going on vacation',
  'i go on vacation']}

In [21]:
prompt_response_map[training_csv[3].split('\t')[1]][0]

'a junior suite'

In [32]:
def classify_language_meaning(grammar, line, activate_transcript=False):
    split_line = line.split("\t")
    # key for prompt reponse map
    prompt_id = split_line[1]
    rec_result = split_line[2]
    transcript = split_line[3]
    if prompt_id not in  grammar:
        return False
    responses = grammar[prompt_id]
    #responses = prompt_response_map[prompt_id]
    if activate_transcript:
        for response in responses:
            if rec_result == response or transcript == response:
                return True
    else:
        for response in responses:
            if rec_result == response:
                    return True
        return False 

In [33]:
def quick_rg_test_analytics(data_set, grammar):
    # RG + Language + Meaning
    intersection_rg_lang_meaning = []
    # RG + Language
    intersection_rg_lang = []
    # RG + Meaning
    intersection_rg_meaning = []
    # Lang + Meaning // can be append
    intersection_lang_meaning = []
    # Lang
    lang = []
    # Meaning 
    mean = []
    # Inside Reference Grammar
    rg_correct = []
    # Missing in Reference Grammar
    rg_missing = []
    dif_intersection = []
    for sample in data_set:
        split = sample.split("\t")
        language = split[4]
        meaning = split[5].replace("\n", "")
        if classify_language_meaning(grammar, sample) == True:
            if language == "correct" and meaning == "correct":
                intersection_rg_lang_meaning.append(sample)
            elif language == "correct":
                intersection_rg_lang.append(sample)
            elif meaning == "correct":
                intersection_rg_meaning.append(sample)
            else:
                rg_correct.append(sample)
        elif language == "correct" and meaning == "correct":
            intersection_lang_meaning.append(sample)
        elif language == "correct":
            lang.append(sample)
        elif meaning == "correct":
            mean.append(sample)
        else:
            rg_missing.append(sample)
    print(len(training_csv[1:]))
    print("Correct through RG: %s \t False through RG: %s" % (len(rg_correct), len(rg_missing)))
    print("Intersection RG, Lang, Meaning: %s" % len(intersection_rg_lang_meaning))
    print("Intersection RG, Lang : %s" % len(intersection_rg_lang))
    print("Intersection RG, Meaning : %s" % len(intersection_rg_meaning))
    print("___________")
    print("Language, Meaning: %s" % len(intersection_lang_meaning))
    print("Language: %s" % len(lang))
    print("Meaning: %s" % len(mean))        
    return intersection_rg_lang_meaning, intersection_rg_lang, intersection_rg_meaning,\
        intersection_lang_meaning, lang, mean, \
    rg_correct, rg_missing, dif_intersection 


intersection_rg_lang_meaning,intersection_rg_lang, intersection_rg_meaning, \
intersection_lang_meaning, lang, mean, \
rg_correct, rg_missing, dif_intersection = quick_rg_test_analytics(training_csv[1:], prompt_response_map)



5222
Correct through RG: 20 	 False through RG: 520
Intersection RG, Lang, Meaning: 2154
Intersection RG, Lang : 0
Intersection RG, Meaning : 54
___________
Language, Meaning: 1726
Language: 0
Meaning: 748


## Add Diff Reference Grammar

In [25]:
diff_prompts = {}
for line in intersection_lang_meaning:
    split = line.split('\t')
    key = split[1]
    transcript = split[3]
    if key in diff_prompts:
        diff_prompts[key].append(transcript)
    else:
        arr = []
        arr.append(transcript)
        diff_prompts[key] = arr

### Uniques

In [26]:
diff_unique = {}
for key in diff_prompts:
    diff_unique[key] = set(diff_prompts[key])

In [27]:
diff_unique

{'Frag: 1 Musical-Ticket': {'can i have one musical ticket',
  'could i have one musical ticket',
  'do you have one musical ticket',
  'i need one musical ticket',
  'i want one musical ticket',
  'i would like one musical ticket',
  "i'd like one ticket for the musical",
  'one musical ticket'},
 'Frag: 1 Ticket': {'i need one ticket', 'i would like one ticket ticket'},
 'Frag: 1 Ticket für das Phantom der Oper': {'a ticket for the phantom of the opera'},
 'Frag: 2 Musical-Tickets': {'i want two musical tickets',
  'i would like two musical tickets',
  'i would like two tickets for the musical'},
 'Frag: 2 Tickets': {'can i have two tickets',
  'i need two tickets',
  'two tickets'},
 'Frag: 2 Tickets für Mittwochabend': {'two tickets for wednesday night'},
 'Frag: 2 Tickets für Wicked': {'i want two tickets for wicked'},
 'Frag: 2 Tickets für heute Abend': {'i want two tickets for tonight'},
 'Frag: 2 Tickets nach London': {'can i have two tickets to london',
  'i would like two tic

In [28]:
def create_diff_xml(filename, mapping):
    root = ET.Element("grammar")

    for key in mapping:
        prompt_unit = ET.SubElement(root, 'prompt_unit')
        ET.SubElement(prompt_unit, "prompt").text = key
        for item in mapping[key]:
            ET.SubElement(prompt_unit, "response").text = item

    tree = ET.ElementTree(root)
    tree.write(filename)
    

create_diff_xml("diff_rg_1.xml", diff_unique)

In [29]:
prompt_response_map_2 = read_grammar_and_create_map("diff_rg_1.xml")

In [30]:
prompt_response_map_2

{'Frag: 1 Musical-Ticket': ['i want one musical ticket',
  'i would like one musical ticket',
  "i'd like one ticket for the musical",
  'i need one musical ticket',
  'one musical ticket',
  'can i have one musical ticket',
  'do you have one musical ticket',
  'could i have one musical ticket'],
 'Frag: 1 Ticket': ['i need one ticket', 'i would like one ticket ticket'],
 'Frag: 1 Ticket für das Phantom der Oper': ['a ticket for the phantom of the opera'],
 'Frag: 2 Musical-Tickets': ['i would like two tickets for the musical',
  'i want two musical tickets',
  'i would like two musical tickets'],
 'Frag: 2 Tickets': ['can i have two tickets',
  'two tickets',
  'i need two tickets'],
 'Frag: 2 Tickets für Mittwochabend': ['two tickets for wednesday night'],
 'Frag: 2 Tickets für Wicked': ['i want two tickets for wicked'],
 'Frag: 2 Tickets für heute Abend': ['i want two tickets for tonight'],
 'Frag: 2 Tickets nach London': ['can i have two tickets to london',
  'two tickets to euh b

In [34]:
def quick_rg_test_analytics(data_set, grammar1, grammar2):
    # RG + Language + Meaning
    intersection_rg_lang_meaning = []
    # RG + Language
    intersection_rg_lang = []
    # RG + Meaning
    intersection_rg_meaning = []
    # Lang + Meaning // can be append
    intersection_lang_meaning = []
    # Lang
    lang = []
    # Meaning 
    mean = []
    # Inside Reference Grammar
    rg_correct = []
    # Missing in Reference Grammar
    rg_missing = []
    dif_intersection = []
    for sample in data_set:
        split = sample.split("\t")
        language = split[4]
        meaning = split[5].replace("\n", "")
        if classify_language_meaning(grammar1, sample) == True:
            if language == "correct" and meaning == "correct":
                intersection_rg_lang_meaning.append(sample)
            elif language == "correct":
                intersection_rg_lang.append(sample)
            elif meaning == "correct":
                intersection_rg_meaning.append(sample)
            else:
                rg_correct.append(sample)
        elif classify_language_meaning(grammar2, sample) == True:
            if language == "correct" and meaning == "correct":
                intersection_rg_lang_meaning.append(sample)
            elif language == "correct":
                intersection_rg_lang.append(sample)
            elif meaning == "correct":
                intersection_rg_meaning.append(sample)
            else:
                rg_correct.append(sample)
        elif language == "correct" and meaning == "correct":
            intersection_lang_meaning.append(sample)
        elif language == "correct":
            lang.append(sample)
        elif meaning == "correct":
            mean.append(sample)
        else:
            rg_missing.append(sample)
    print(len(training_csv[1:]))
    print("Correct through RG: %s \t False through RG: %s" % (len(rg_correct), len(rg_missing)))
    print("Intersection RG, Lang, Meaning: %s" % len(intersection_rg_lang_meaning))
    print("Intersection RG, Lang : %s" % len(intersection_rg_lang))
    print("Intersection RG, Meaning : %s" % len(intersection_rg_meaning))
    print("___________")
    print("Language, Meaning: %s" % len(intersection_lang_meaning))
    print("Language: %s" % len(lang))
    print("Meaning: %s" % len(mean))        
    return intersection_rg_lang_meaning, intersection_rg_lang, intersection_rg_meaning,\
        intersection_lang_meaning, lang, mean, \
    rg_correct, rg_missing, dif_intersection 


intersection_rg_lang_meaning,intersection_rg_lang, intersection_rg_meaning, \
intersection_lang_meaning, lang, mean, \
rg_correct, rg_missing, dif_intersection = quick_rg_test_analytics(training_csv[1:], prompt_response_map, prompt_response_map_2)



5222
Correct through RG: 25 	 False through RG: 515
Intersection RG, Lang, Meaning: 2799
Intersection RG, Lang : 0
Intersection RG, Meaning : 69
___________
Language, Meaning: 1081
Language: 0
Meaning: 733


In [39]:
"""
with open('out_1.csv', 'w') as writer:
    for sent in rg_missing:
        writer.write(sent)"""

## Pipeline for Filter Double

In [63]:
dump_out = []
with open('out_1.csv', 'r') as reader:
    for sent in reader.readlines():
        dump_out.append(sent)

In [77]:
def filter_double_and_check_rg(prompt):
    split = prompt.split('\t')
    rec_result = split[2]
    words = rec_result.split(" ")
    unique_words = set(words)
    print(split)
    return unique_words
    
filter_double_and_check_rg(dump_out[4])

['5902', 'Frag: ein Sweatshirt', 'i want a suite i want a sweet shirt', 'i want a i want a sweet shirt', 'incorrect', 'incorrect\n']


{'a', 'i', 'shirt', 'suite', 'sweet', 'want'}

## Counting errors

In [80]:
prompt_rec_errors = {}
prompt_transcript_errors = {}

for sent in dump_out:
    split = sent.split("\t")
    prompt_id = split[1]
    rec_result = split[2] 
    transcript = split[3] 
    if prompt_id in prompt_rec_errors:
        prompt_rec_errors[prompt_id].append(rec_result)
        prompt_transcript_errors[prompt_id].append(transcript)
    else:
        arr1 = []
        arr1.append(rec_result)
        prompt_rec_errors[prompt_id] = arr1
        arr2 = []
        arr2.append(transcript)
        prompt_transcript_errors[prompt_id] = arr2

In [86]:
len(prompt_rec_errors['Frag: Wo ist das Hotelrestaurant?'])

4

In [87]:
prompt_id_count_errors = {}
for key in prompt_rec_errors:
    prompt_id_count_errors[key] =     len(prompt_rec_errors[key])

In [91]:
with open('count_errors.csv', 'w') as writer:
    for key in prompt_id_count_errors:
        writer.write(key + "\t" + str(prompt_id_count_errors[key]) + "\n")

In [93]:
set(prompt_rec_errors['Sag: Ich gehe in die Ferien'])

{'*** *** *** ***',
 '*** *** my name is rome',
 "*** i'm going on holiday",
 "*** i'm going on holidays",
 'holidays',
 'i *** *** ***',
 'i *** going on holidays',
 'i am going for holiday',
 'i am going for holidays',
 'i am going into the vacation',
 'i am going on holidays',
 'i am going to the vacation',
 'i go *** into holidays',
 'i go for holidays',
 'i go in the holidays',
 'i go into the holidays',
 'i go on holidays',
 'i go to the holidays',
 'i going into holiday',
 'i going on the holidays',
 'i have a shirt',
 'i went to holidays',
 "i'm going on holiday",
 "i'm going on tuesday wien",
 "i'm going to ***",
 'room for the holidays'}

In [94]:
set(prompt_transcript_errors['Sag: Ich gehe in die Ferien'])

{'holidays',
 'i am going for holidays',
 'i am going for the holidays',
 'i am going on holiday',
 'i am going on holidays',
 'i am going to the vacation',
 'i go for holidays',
 'i go in holiday',
 'i go in the holidays',
 'i go into the holidays',
 'i go to holidays',
 'i go to the holiday',
 'i go to the hollywood',
 'i going into holiday',
 'i going on the ferien',
 'i going on the holidays',
 'i going to holdays',
 'i going to holidays',
 'i went to holidays',
 "i'm going on holiday",
 "i'm going to holly",
 'in two have holidays'}

In [102]:
unique_prompt_ferien = set(prompt_transcript_errors['Sag: Ich gehe in die Ferien'])
for item in unique_prompt_ferien:
    doc1 = nlp_en(u''+item)
    sentence = item + "\t"
    for word in doc1:
        sentence += "(" + word.tag_ +"," +word.pos_ + ")"
    print(sentence)

i am going for holidays	(PRP,PRON)(VBP,VERB)(VBG,VERB)(IN,ADP)(NNS,NOUN)
i going on the holidays	(PRP,PRON)(VBG,VERB)(IN,ADP)(DT,DET)(NNS,NOUN)
i go for holidays	(PRP,PRON)(VBP,VERB)(IN,ADP)(NNS,NOUN)
i going on the ferien	(PRP,PRON)(VBG,VERB)(IN,ADP)(DT,DET)(NN,NOUN)
i went to holidays	(PRP,PRON)(VBD,VERB)(IN,ADP)(NNS,NOUN)
i going into holiday	(PRP,PRON)(VBG,VERB)(IN,ADP)(NN,NOUN)
i going to holidays	(PRP,PRON)(VBG,VERB)(IN,ADP)(NNS,NOUN)
i go in holiday	(PRP,PRON)(VBP,VERB)(IN,ADP)(NN,NOUN)
i am going on holiday	(PRP,PRON)(VBP,VERB)(VBG,VERB)(IN,ADP)(NN,NOUN)
i am going on holidays	(PRP,PRON)(VBP,VERB)(VBG,VERB)(IN,ADP)(NNS,NOUN)
i go into the holidays	(PRP,PRON)(VBP,VERB)(IN,ADP)(DT,DET)(NNS,NOUN)
i go to the hollywood	(PRP,PRON)(VBP,VERB)(IN,ADP)(DT,DET)(NN,NOUN)
i'm going to holly	(PRP,PRON)(VBP,VERB)(VBG,VERB)(TO,PART)(RB,ADV)
i am going to the vacation	(PRP,PRON)(VBP,VERB)(VBG,VERB)(IN,ADP)(DT,DET)(NN,NOUN)
holidays	(NNS,NOUN)
i go to the holiday	(PRP,PRON)(VBP,VERB)(IN,ADP)(DT

In [103]:
#prompt_response_map = read_grammar_and_create_map('man_rg.xml')


In [116]:
import re
def nlp_it_gen(sentence):
    pattern = re.compile(r"\((.*?)\)")
    result = pattern.findall(sentence)
    print(result)

nlp_it_gen(prompt_response_map['Sag: Ich gehe in die Ferien'][0])
prompt_response_map['Sag: Ich gehe in die Ferien'][0]

["am|'m|EMPTY", 'holidays|holiday']


"i (am|'m|EMPTY) going ADPDET (holidays|holiday)"

In [ ]:
i (GO VERB) (PP**) (holiday|holidays)
i (GO) on ho

## MERGE DICTIONARIES OF RG

In [437]:
import copy
def merge_response_map(grammar1, grammar2):
    grammar = copy.deepcopy(grammar1)
    for key in grammar1:
        if key in grammar2:            
            for item in grammar2[key]:
                arr = list(grammar[key])
                arr.append(item)
                grammar[key] = arr
            grammar[key] = set(grammar[key])
    return grammar

In [438]:
prompt_response_map_1 = read_grammar_and_create_map('referenceGrammar.xml')
prompt_response_map_2 = read_grammar_and_create_map('diff_rg_1.xml')
prompt_response_map_3 = read_grammar_and_create_map('man_rg.xml')

rg_total = merge_response_map(prompt_response_map_1, prompt_response_map_2)
rg_total = merge_response_map(rg_total, prompt_response_map_3)

In [439]:
prompt_response_map_3 = read_grammar_and_create_map('man_rg.xml')
rg_total['Sag: Ich gehe in die Ferien']

{'i am going on vacation',
 'i am on holiday',
 'i am on holidays',
 'i go on vacation',
 "i'm going on vacation",
 "i'm on holiday",
 "i'm on holidays"}

In [411]:
key1 = "Frag: Pommes Frittes"

print(len(rg_total[key1]))
print(len(prompt_response_map_1[key1]))
print(len(prompt_response_map_2[key1]))
print("__________")

key2 = "Sag: Ich gehe in die Ferien"
print(len(rg_total[key2]))
print(len(prompt_response_map_1[key2]))
print(len(prompt_response_map_2[key2]))
print(len(prompt_response_map_3[key2]))

101
100
2
__________
4
2
3
0


## LEMMAS GRAMMAR

In [317]:
from spacy.en import English
parser = English()

In [340]:
def lemma_grammar(data_grammar):
    
    lemma_prompt = {}
    for prompt in data_grammar:
        lemma_prompt[prompt] = []
        for sentence in data_grammar[prompt]:
            parsed = parser(sentence)
            lemmas = []
            for i, token in enumerate(parsed):
                lemma = token.lemma_
                lemmas.append(lemma)

            lemma_prompt[prompt].append(" ".join(lemmas))
        #print(len(lemma_prompt[prompt]))
        lemma_prompt[prompt] = set(lemma_prompt[prompt])
    return lemma_prompt

lemma_gram = lemma_grammar(prompt_response_map)

In [341]:
create_diff_xml("lemma_gram_dif.xml", lemma_gram)

In [358]:
only_lemma_gram = {}
for key in lemma_gram:
    if key in prompt_response_map:
        for item in lemma_gram[key]:
            if item not in prompt_response_map[key]:
                if key in only_lemma_gram:
                    only_lemma_gram[key].append(item)
                else:
                    arr = []
                    arr.append(item)
                    only_lemma_gram[key] = arr

                    #len(lemma_gram)
#create_diff_xml("lemma_gram_dif_small.xml", C)

454

In [359]:
create_diff_xml("lemma_gram_dif_small.xml", only_lemma_gram)

In [351]:
def classify_via_lemma(grammar, line, activate_transcript=False):
    split_line = line.split("\t")
    # key for prompt reponse map
    prompt_id = split_line[1]
    rec_result = split_line[2]
    transcript = split_line[3]
    if prompt_id not in  grammar:
        return False
    responses = grammar[prompt_id]
    #responses = prompt_response_map[prompt_id]

    for response in responses:
        rec_result = " ".join(ferien_nlp_pipeline(rec_result)[1])
        if rec_result == response:
                return True
    return False 
    
def quick_test_lemma(data_set, grammar):
    # RG + Language + Meaning
    intersection_rg_lang_meaning = []
    # RG + Language
    intersection_rg_lang = []
    # RG + Meaning
    intersection_rg_meaning = []
    # Lang + Meaning // can be append
    intersection_lang_meaning = []
    # Lang
    lang = []
    # Meaning 
    mean = []
    # Inside Reference Grammar
    rg_correct = []
    # Missing in Reference Grammar
    rg_missing = []
    dif_intersection = []
    for sample in data_set:
        split = sample.split("\t")
        language = split[4]
        meaning = split[5].replace("\n", "")
        if classify_via_lemma(grammar, sample) == True:
            if language == "correct" and meaning == "correct":
                intersection_rg_lang_meaning.append(sample)
            elif language == "correct":
                intersection_rg_lang.append(sample)
            elif meaning == "correct":
                intersection_rg_meaning.append(sample)
            else:
                rg_correct.append(sample)
        elif language == "correct" and meaning == "correct":
            intersection_lang_meaning.append(sample)
        elif language == "correct":
            lang.append(sample)
        elif meaning == "correct":
            mean.append(sample)
        else:
            rg_missing.append(sample)
    print(len(training_csv[1:]))
    print("Correct through RG: %s \t False through RG: %s" % (len(rg_correct), len(rg_missing)))
    print("Intersection RG, Lang, Meaning: %s" % len(intersection_rg_lang_meaning))
    print("Intersection RG, Lang : %s" % len(intersection_rg_lang))
    print("Intersection RG, Meaning : %s" % len(intersection_rg_meaning))
    print("___________")
    print("Language, Meaning: %s" % len(intersection_lang_meaning))
    print("Language: %s" % len(lang))
    print("Meaning: %s" % len(mean))        

quick_test_lemma(training_csv[1:], C)

5222
Correct through RG: 24 	 False through RG: 516
Intersection RG, Lang, Meaning: 2642
Intersection RG, Lang : 0
Intersection RG, Meaning : 95
___________
Language, Meaning: 1238
Language: 0
Meaning: 707


# NLP FIX

In [ ]:
from spacy.en import English
parser = English()

In [393]:
def tag_pos_sentence(sentence):
    doc_en = nlp_en(sentence)        
    tag_en = ""
    pos_en = ""

    for word in doc_en:
        if word.orth_.isspace() == False:
            tag_en += word.tag_ + ' '
            pos_en  += word.pos_ + ' '
    return tag_en, pos_en

def nlp_sentence(sentence):
    parsed = parser(sentence)
    lemmas = []
    words = []
    tags = []
    poss = []
    for i, token in enumerate(parsed):
        lemma = token.lemma_
        words.append(token)
        lemmas.append(lemma)
        tag = token.tag_
        pos = token.tag_
        tags.append(tag)
        poss.append(pos)
    return words, lemmas, tags, poss
    
def filter_noise(items):
    rec_result_filtered = []

    for item in items:
        append_to = True
        words, lemmas, tags, poss = nlp_sentence(item["rec_result"])
        for word in words:
            if str(word) == "*":
                append_to = False
                break
        if append_to:
            nlp_item = {"rec_result": item["rec_result"],"words":words, "lemmas": lemmas, "tags": tags, "poss": poss}
            rec_result_filtered.append(nlp_item)

    return rec_result_filtered

In [394]:
rec_result_set, transcript_set = prompt_csv_map(training_csv[1:])  

## NLP GRAMMAR

In [462]:
import re

In [468]:
pos_pattern = r"\[(.*?)\]"
pos_regex = re.compile(pos_pattern)

In [497]:
word_choice_pattern = r"\((.*?)\)"
word_choice_regex = re.compile(word_choice_pattern)

In [568]:
def create_items_ferien():
    example = "i go [*EMP|*IN|*DT|*IN*&*DT] (holiday|holidays|vacation)"
    split_string = example.split(" ")
    print(split_string)
    #for item in 
    items = []
    for item in split_string:
        pos_matches = re.findall(pos_regex, item)
        word_choice_matches = re.findall(word_choice_regex, item)

        if len(pos_matches) == 1:
            pos_items = "".join(pos_matches).replace("[", "").replace("]", "").replace("*", "").split("|")
            pos_opts = []
            for pos_item in pos_items:
                if "&" in pos_item:
                    and_string = pos_item.replace("&", " ").split(" ")
                    key = 'POS'
                    arr = []
                    for under_pos in and_string:
                        arr.append
                    pos_opts.append(and_string)
                else:
                    pos_opts.append(pos_item)
            items.append({"POS": pos_opts})
        elif len(word_choice_matches) == 1:
            word_choice_matches_items = "".join(word_choice_matches).replace("(", "").replace(")", "").split("|")
            words = []
            for word_item in word_choice_matches_items:
                words.append(word_item)
            items.append({"WORDS": words})
        else: 
            items.append(item)
    return items
#print(re.findall(regex, split_string[0]))

In [569]:
items = create_items_ferien()
items

['i', 'go', '[*EMP|*IN|*DT|*IN*&*DT]', '(holiday|holidays|vacation)']


['i',
 'go',
 {'POS': ['EMP', 'IN', 'DT', ['IN', 'DT']]},
 {'WORDS': ['holiday', 'holidays', 'vacation']}]

In [490]:
example = "i go [*EMP|*IN|*DT|*IN*&*DT] (holiday|holidays|vacation)"
split_string = example.split(" ")
for item in split_string:
    print(item)

i
go
[*EMP|*IN|*DT|*IN*&*DT]
(holiday|holidays|vacation)


In [477]:
string = "[*EMP|*IN|*DT|*IN & *DT]"
string = string.replace("[", "").replace("]", "").replace("*", "")
string.split("|")
"IN "

['EMP', 'IN', 'DT', 'IN & DT']

### Sag: Ich gehe in die Ferien

In [441]:
key = "Sag: Ich gehe in die Ferien"

[{'language': 'correct',
  'meaning': 'correct',
  'rec_result': 'i am on holiday',
  'transcript': 'i am on holiday'},
 {'language': 'correct',
  'meaning': 'correct',
  'rec_result': 'i am on holiday',
  'transcript': 'i am on holiday'},
 {'language': 'correct',
  'meaning': 'correct',
  'rec_result': 'i am on holiday',
  'transcript': "i'm on holiday"},
 {'language': 'correct',
  'meaning': 'correct',
  'rec_result': 'i am on holidays',
  'transcript': 'i am on holidays'},
 {'language': 'incorrect',
  'meaning': 'incorrect',
  'rec_result': 'holidays',
  'transcript': 'holidays'},
 {'language': 'correct',
  'meaning': 'correct',
  'rec_result': 'i am on holidays',
  'transcript': 'i am on holidays'},
 {'language': 'correct',
  'meaning': 'correct',
  'rec_result': 'i am on holidays',
  'transcript': 'i am on holidays'},
 {'language': 'correct',
  'meaning': 'correct',
  'rec_result': "*** i'm on holidays",
  'transcript': 'i am on holidays'},
 {'language': 'correct',
  'meaning': 'c

In [444]:
key = "Sag: Ich gehe in die Ferien"
#rec_result_set[key]
def filter_correct_incorrect(data_set):
    both_correct_items = []
    meaning_false_items = []
    language_false_items = []
    for item in data_set:
        if item['meaning'] == 'correct' and item['language'] == 'correct':
            both_correct_items.append(item)
        elif item['meaning'] == 'incorrect':
            meaning_false_items.append(item)
        elif item['language'] == 'incorrect':
            language_false_items.append(item)
    return both_correct_items, meaning_false_items, language_false_items
        
both_correct_ferien, meaning_false_ferien, language_false_ferien = filter_correct_incorrect(rec_result_set[key])
print("Both correct: %s " % len(both_correct_ferien))
print("Meaning incorrect: %s " % len(meaning_false_ferien))
print("Language incorrect: %s " % len(language_false_ferien))

Both correct: 18 
Meaning incorrect: 50 
Language incorrect: 20 


In [445]:
meaning_false_filtered = filter_noise(meaning_false_ferien)
language_false_filtered = filter_noise(language_false_ferien)
both_correct_filtered = filter_noise(both_correct_ferien)
print("Filtered Both correct: %s " % len(both_correct_filtered))
print("Filtered Meaning incorrect: %s " % len(meaning_false_filtered))
print("Filtered Language incorrect: %s " % len(language_false_filtered))

Filtered Both correct: 17 
Filtered Meaning incorrect: 36 
Filtered Language incorrect: 16 


In [ ]:
grammar = rg_total["Sag: Ich gehe in die Ferien"]
for item in meaning_false_filtered:
    print(item["rec_result"])
    #print(item["poss"])

In [590]:
def create_word_occur_map(words):
    word_occur = {}
    index =0
    for word in words:
        word_occur[str(index) + str(word)] = False
        index += 1
    return word_occur

def meaning_correct(word_map):
    for item in word_map:
        if word_map[item] == True:
            continue
        else:
            return False
    return True

In [598]:
items = create_items_ferien()
test_string = "i going for holiday"
words, lemmas, tags, poss = nlp_sentence(test_string)
print(words)
test_string_split = test_string.split(" ")
words_map = create_word_occur_map(words)
word_counter = 0
for word in words:
    print("Current Word:" + str(word) + " \t" + str(counter))
    ## word checking
    
    pos_word = poss[word_counter]
    key = str(word_counter) + str(word)
    for counter in range(0, len(items)):
        # Remove items
        if str(word) == items[counter] or str(lemmas[word_counter]) == items[counter]:
            #print("Word removed")
            words_map[key] = True
            break
        elif "POS" in items[counter]:
            print(items[counter]["POS"])
            if pos_word in items[counter]["POS"]:
                words_map[key] = True
                print(str(word) +" True")
        elif "WORDS" in items[counter]:
            if str(word) in items[counter]["WORDS"]:
                words_map[key] = True

    word_counter += 1
meaning_correct(words_map)

['i', 'go', '[*EMP|*IN|*DT|*IN*&*DT]', '(holiday|holidays|vacation)']
[i, going, for, holiday]
Current Word:i 	3
Current Word:going 	0
Current Word:for 	1
['EMP', 'IN', 'DT', ['IN', 'DT']]
for True
Current Word:holiday 	3
['EMP', 'IN', 'DT', ['IN', 'DT']]


True

In [595]:
test_string = "i am going for holiday"
words, lemmas, tags, poss = nlp_sentence(test_string)
print(words)
test_string_split = test_string.split(" ")
counter = 0


for symbol in items:
    current_item = test_string_split[counter]
    print("%s \t %s" % (symbol, current_item))
    if current_item == symbol:
        print("Equal TRUE")
    else:
        if 'POS' in symbol:
            #print(test_string_nlp)
            if poss[counter] in symbol['POS']:
                print("POS TRUE")

        elif 'WORDS' in symbol:
            word = test_string_split[counter]
            if word in symbol['WORDS']:
                print("WORDS TRUE")
        else:
            print("FALSE")

    counter += 1
    #if current_item == symbol:


[i, am, going, for, holiday]
i 	 i
Equal TRUE
go 	 am
FALSE
{'POS': ['EMP', 'IN', 'DT', ['IN', 'DT']]} 	 going
{'WORDS': ['holiday', 'holidays', 'vacation']} 	 for


In [ ]:
dict_pos_map = {}
dict_tag_map = {}
for item in both_correct_filtered:
    #for pos in item["poss"]:
    words = item["words"]
    pos_counter = 0
    for pos in item["poss"]:
        word = words[pos_counter]
        pos_counter += 1
        if pos in dict_pos_map:
            dict_pos_map[pos].append(word)
        else:
            arr = []
            arr.append(word)
            dict_pos_map[pos] = arr
    tag_counter = 0
    for tag in item["tags"]:
        word = words[tag_counter]
        tag_counter += 1
        if tag in dict_tag_map:
            dict_tag_map[tag].append(word)
        else:
            arr = []
            arr.append(word)
            dict_tag_map[tag] = arr
    print(item["words"])
    print(item["poss"])
    print(item["tags"])

In [343]:
def ferien_nlp_pipeline(sentence):
    
    return words, lemmas
print()
ferien_nlp_pipeline(both_correct[6]['rec_result'])
ferien_nlp_pipeline("I went to vacation forty-two")
#ferien_nlp_pipeline("no could you give me red trouser please")
#ferien_nlp_pipeline("thank you could you please offer me red trouser")

'i go to vacation forty - two'

In [594]:
aray = []
for item in meaning_false:
    #print(item["rec_result"])
    aray.append(item["rec_result"])
    #print(tag_pos_sentence(item["rec_result"])[0])
set(aray)

{'*** *** *** ***',
 '*** *** my name is rome',
 "*** i'm going on holiday",
 "*** i'm going on holidays",
 'holidays',
 'i *** *** ***',
 'i *** going on holidays',
 'i am going for holiday',
 'i am going for holidays',
 'i am going into the vacation',
 'i am going on holidays',
 'i am going to the vacation',
 'i go *** into holidays',
 'i go for holidays',
 'i go in the holidays',
 'i go into the holidays',
 'i go on holidays',
 'i go to the holidays',
 'i going into holiday',
 'i going on the holidays',
 'i have a shirt',
 'i went to holidays',
 "i'm going on holiday",
 "i'm going on tuesday wien",
 "i'm going to ***",
 'room for the holidays'}

In [ ]:
def create_possible_items():
    example = "i go [*EMP|*IN|*DT|*IN*&*DT] (holiday|holidays|vacation)"
    split_string = example.split(" ")
    print(split_string)
    #for item in 
    items = []
    for item in split_string:
        pos_matches = re.findall(pos_regex, item)
        word_choice_matches = re.findall(word_choice_regex, item)

        if len(pos_matches) == 1:
            pos_items = "".join(pos_matches).replace("[", "").replace("]", "").replace("*", "").split("|")
            pos_opts = []
            for pos_item in pos_items:
                if "&" in pos_item:
                    and_string = pos_item.replace("&", " ").split(" ")
                    key = 'POS'
                    arr = []
                    for under_pos in and_string:
                        arr.append
                    pos_opts.append(and_string)
                else:
                    pos_opts.append(pos_item)
            items.append({"POS": pos_opts})
        elif len(word_choice_matches) == 1:
            word_choice_matches_items = "".join(word_choice_matches).replace("(", "").replace(")", "").split("|")
            words = []
            for word_item in word_choice_matches_items:
                words.append(word_item)
            items.append({"WORDS": words})
        else: 
            items.append(item)
    return items

create_possible_items()

In [689]:
def add_elements(base, elements, counter):
    if len(elem) == 0:
        return base
    base[len(base)-counter] = base[len(base)-counter] + " "+ elements[0]
    counter -= 1
    return add_base(base, elements[1:], counter)

def base_it(new_arr, base, elements):
    if len(elements) == 0:
        return new_arr
    new_arr.extend(base)
    return base_it(new_arr, base, elements[1:])

    
elements = ["a", "b"]
base = ["i go"]
base_ = ["i go", "i go"]
#permuate(base_, array_)
add_base(["i go", "i go"], ["a", "b"], len(base_))
add_base(base_it([], base, elements), elements, 2)
# new_base(["i go"], ["a", "b"]) => I go a, I go b

['i go a', 'i go b']

In [ ]:
def rec_match(array, base):
    if len(array) == 0:
        return base
    array2 = array[0].split("|")   

    if len(array2) > 1:
        new_base =[]
        for i in range(0, len(base)):
            pair = []
            for j in array2:
                pair.append(base[i])
            new_base.append(pair)
        
        for item in new_base:
            print("item")
            print(item)
            for index in range(0, len(item)-1):
                new_base.append(str(item[index]) + " " + array2[index])
        
        if len(array) == 1:
            return rec_match(array[0], new_base)
        return rec_match(array[1:], new_base)

    for item in range(0, len(base)):
        base[item] = base[item] + array[0] + " "
    return rec_match(array[1:], base)
        

split_arr = "i go wrong|right".split(" ")
#split_arr = "i go".split(" ")
rec_match(split_arr, [""])

In [606]:
string_eample = "i go wrong|right ".split(" ")

base = ""
for item in string_eample:
    split = item.split("|")
    if len(split) == 1:
        base += split[0] + " "
    else:
        for item2 in split:
            print(base + item2)
        
    print(base)

i 
i go 
i go wrong
i go right
i go 
